# ABOUT
- this code performs aggregation and feature engineering on elo merchant data
    - taken from manoj
    - should be understood and "paraphrased"
    - takes to long to process
    - processed data can just be downloaded from manoj too

In [ ]:
import pandas as pd
import datetime
import gc
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:


train_path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\train.csv"
test_path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\test.csv"
merchants_path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\merchants.csv"
historical_transactions_path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\historical_transactions.csv"
new_transactions_path = r"C:\Users\tanch\Documents\NTU\NTU Year 4\Semester 1\CZ4041 - Machine Learning\Team Project\data\new_merchant_transactions.csv"


feature_names = {
    "train_test":{
        "id":["card_id"],
        "categoric":['feature_1', 'feature_2', 'feature_3'],
        "datetime":["first_active_month"]
    },
    "merchants":{
        "id": ['merchant_id', 'merchant_group_id', 'merchant_category_id','subsector_id', 'city_id', 'state_id'],
        "categoric": ['category_1','most_recent_sales_range', 'most_recent_purchases_range','category_4', 'category_2'],
        "numeric": ['numerical_1', 'numerical_2','avg_sales_lag3', 'avg_purchases_lag3','avg_sales_lag6', 'avg_purchases_lag6','avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag3','active_months_lag6','active_months_lag12']
    },
    "transactions":{
        "id": ['card_id', 'city_id','merchant_category_id', 'merchant_id','state_id','subsector_id'],
        "categoric": ['authorized_flag','category_1',"category_3", "category_2"],
        "numeric": ['purchase_amount', 'month_lag',"installments"],
        "datetime": ['purchase_date']
    },
}


In [ ]:
def featurize_train_test(df):
    # to datetime
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])

    # datetime features
    df['quarter'] = df['first_active_month'].dt.quarter
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days

    df['days_feature1'] = df['elapsed_time'] * df['feature_1']
    df['days_feature2'] = df['elapsed_time'] * df['feature_2']
    df['days_feature3'] = df['elapsed_time'] * df['feature_3']

    df['days_feature1_ratio'] = df['feature_1'] / df['elapsed_time']
    df['days_feature2_ratio'] = df['feature_2'] / df['elapsed_time']
    df['days_feature3_ratio'] = df['feature_3'] / df['elapsed_time']


    df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
    df['feature_mean'] = df['feature_sum']/3
    df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
    df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
    df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

    t1 = pd.get_dummies(df.feature_1, prefix = 'feature1')
    t2 = pd.get_dummies(df.feature_2, prefix = 'feature2')
    t3 = pd.get_dummies(df.feature_3, prefix = 'feature3')

    df[t1.columns] = t1
    df[t2.columns] = t2
    df[t3.columns] = t3

    del t1, t2, t3
    gc.collect()

    return df

In [ ]:
def Featurized(df1, prefix_string):
    flag = 0
    df1['authorized_flag'] = df1['authorized_flag'].map({'Y':1, "N":0})   # categorical encoding
    df1['category_1'] = df1['category_1'].map({'Y':1, "N":0})

    df1['category_2'].fillna(1.0,inplace=True)                  # na filling 
    df1['category_3'].fillna('A',inplace=True)
    df1['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    df1['installments'].replace(-1, np.nan,inplace=True)
    df1['installments'].replace(999, np.nan,inplace=True)

    gb = df1.groupby('card_id')

    df_features = pd.DataFrame()

    df_features['hist_count'] = gb['card_id'].count()

    #authorized_flag - groupby card_id and compute count and fraction of each class
    df_authorized_flag_count = df1.groupby('card_id')['authorized_flag'].value_counts().unstack()
    df_authorized_flag_fraction = np.divide(df_authorized_flag_count, df_authorized_flag_count.sum(axis = 1).values.reshape(-1,1))
    df_authorized_flag_count.columns = df_authorized_flag_count.columns.name +'__' +df_authorized_flag_count.columns.astype('str')+ '_count'
    df_authorized_flag_fraction.columns = df_authorized_flag_fraction.columns.name +'__' +df_authorized_flag_fraction.columns.astype('str')+'_fraction'


    # 'category_1'
    df_category1_count = df1.groupby('card_id')['category_1'].value_counts().unstack()
    df_category1_fraction = np.divide(df_category1_count, df_category1_count.sum(axis = 1).values.reshape(-1,1))
    df_category1_count.columns = df_category1_count.columns.name +'__' +df_category1_count.columns.astype('str')+ '_count'
    df_category1_fraction.columns = df_category1_fraction.columns.name +'__' +df_category1_fraction.columns.astype('str')+'_fraction'


    # 'category_2'
    df_category2_count = df1.groupby('card_id')['category_2'].value_counts().unstack()
    df_category2_fraction = np.divide(df_category2_count, df_category2_count.sum(axis = 1).values.reshape(-1,1))
    df_category2_count.columns = df_category2_count.columns.name +'__' +df_category2_count.columns.astype('str')+ '_count'
    df_category2_fraction.columns = df_category2_fraction.columns.name +'__' +df_category2_fraction.columns.astype('str')+'_fraction'
    
     # 'category_3'
    df_category3_count = df1.groupby('card_id')['category_3'].value_counts().unstack()
    df_category3_fraction = np.divide(df_category3_count, df_category3_count.sum(axis = 1).values.reshape(-1,1))
    df_category3_count.columns = df_category3_count.columns.name +'__' +df_category3_count.columns.astype('str')+ '_count'
    df_category3_fraction.columns =  df_category3_fraction.columns.name +'__' + df_category3_fraction.columns.astype('str')+'_fraction'


    #'city_id'
        #bin creation
    city_id_count = df1.groupby('city_id')['city_id'].count()
    np.log(city_id_count).hist()
    bins = pd.qcut(np.log(city_id_count), 5, duplicates='drop')
    df1['city_id_bins'] = df1['city_id'].map(bins)

        #column creation (count and fraction)
    df_city_id_count = df1.groupby('card_id')['city_id_bins'].value_counts().unstack()
    df_city_id_fraction = np.divide(df_city_id_count, df_city_id_count.sum(axis = 1).values.reshape(-1,1))
    df_city_id_count.columns = df_city_id_count.columns.name +'__' +df_city_id_count.columns.astype('str')+ '_count'
    df_city_id_fraction.columns = df_city_id_fraction.columns.name +'__' + df_city_id_fraction.columns.astype('str')+'_fraction'


    #'merchant_category_id'
        #bin creation
    merchant_category_id_count = df1.groupby('merchant_category_id')['merchant_category_id'].count()
    np.log(merchant_category_id_count).hist()
    bins = pd.qcut(np.log(merchant_category_id_count), 5,duplicates='drop')
    df1['merchant_category_id_bins'] = df1['merchant_category_id'].map(bins)

        #column creation (count and fraction)
    df_merchant_category_id_count = df1.groupby('card_id')['merchant_category_id_bins'].value_counts().unstack()
    df_merchant_category_id_fraction = np.divide(df_merchant_category_id_count, df_merchant_category_id_count.sum(axis = 1).values.reshape(-1,1))
    df_merchant_category_id_count.columns = df_merchant_category_id_count.columns.name +'__' +df_merchant_category_id_count.columns.astype('str')+ '_count'
    df_merchant_category_id_fraction.columns = df_merchant_category_id_fraction.columns.name +'__' + df_merchant_category_id_fraction.columns.astype('str')+'_fraction'

    #'merchant_id'
        #bin creation
    merchant_id_count = df1.groupby('merchant_id')['merchant_id'].count()
    np.log(merchant_id_count).hist()
    bins = pd.qcut(np.log(merchant_id_count), 5,duplicates='drop')
    df1['merchant_id_bins'] = df1['merchant_id'].map(bins)

        #column creation (count and fraction)
    df_merchant_id_count = df1.groupby('card_id')['merchant_id_bins'].value_counts().unstack()
    df_merchant_id_fraction = np.divide(df_merchant_id_count, df_merchant_id_count.sum(axis = 1).values.reshape(-1,1))
    df_merchant_id_count.columns = df_merchant_id_count.columns.name +'__' +df_merchant_id_count.columns.astype('str')+ '_count'
    df_merchant_id_fraction.columns = df_merchant_id_fraction.columns.name +'__' + df_merchant_id_fraction.columns.astype('str')+'_fraction'


    #'state_id'
        #bin creation
    state_id_count = df1.groupby('state_id')['state_id'].count()
    np.log(state_id_count).hist()
    bins = pd.qcut(np.log(state_id_count), 5,duplicates='drop')
    df1['state_id_bins'] = df1['state_id'].map(bins)

        #column creation (count and fraction)
    df_state_id_count = df1.groupby('card_id')['state_id_bins'].value_counts().unstack()
    df_state_id_fraction = np.divide(df_state_id_count, df_state_id_count.sum(axis = 1).values.reshape(-1,1))
    df_state_id_count.columns = df_state_id_count.columns.name +'__' +df_state_id_count.columns.astype('str')+ '_count'
    df_state_id_fraction.columns = df_state_id_fraction.columns.name +'__' + df_state_id_fraction.columns.astype('str')+'_fraction'


    #'subsector_id'
        #bin creation
    subsector_id_count = df1.groupby('subsector_id')['subsector_id'].count()
    np.log(subsector_id_count).hist()
    bins = pd.qcut(np.log(subsector_id_count), 5, duplicates='drop')
    df1['subsector_id_bins'] = df1['subsector_id'].map(bins)

        #column creation (count and fraction)
    df_subsector_id_count = df1.groupby('card_id')['subsector_id_bins'].value_counts().unstack()
    df_subsector_id_fraction = np.divide(df_subsector_id_count, df_subsector_id_count.sum(axis = 1).values.reshape(-1,1))
    df_subsector_id_count.columns = df_subsector_id_count.columns.name +'__' +df_subsector_id_count.columns.astype('str')+ '_count'
    df_subsector_id_fraction.columns = df_subsector_id_fraction.columns.name +'__' + df_subsector_id_fraction.columns.astype('str')+'_fraction'

     # 'installments', 'month_lag', 'purchase_amount'

    Min = df1.groupby('card_id')[['installments', 'month_lag', 'purchase_amount']].min()
    Min.columns = np.array(Min.columns)+'_mean'
    Max = df1.groupby('card_id')[['installments', 'month_lag', 'purchase_amount']].max()
    Max.columns = np.array(Max.columns)+'_max'
    Median = df1.groupby('card_id')[['installments', 'month_lag', 'purchase_amount']].median()
    Median.columns = np.array(Median.columns)+'_median'
    Std = df1.groupby('card_id')[['installments', 'month_lag', 'purchase_amount']].std()
    Std.columns = np.array(Std.columns)+'_std'
    Skew = df1.groupby('card_id')[['installments', 'month_lag', 'purchase_amount']].skew()
    Skew.columns = np.array(Skew.columns)+'_skew'
    Mad =df1.groupby('card_id')[['installments', 'month_lag', 'purchase_amount']].mad()
    Mad.columns = np.array(Mad.columns)+'_mad'
    Sum =df1.groupby('card_id')[['installments', 'month_lag', 'purchase_amount']].sum()
    Sum.columns = np.array(Sum.columns)+'_sum'


    # 'purchase_date' 

    df_features["purchase_date_max"] = df1.groupby('card_id')['purchase_date'].max()
    df_features["purchase_date_min"] = df1.groupby('card_id')['purchase_date'].min()

    #df_features['first_buy'] = (df_features['purchase_date_min'] - df['first_active_month']).dt.days

    df1['today_purchase_date'] =  pd.datetime.today() - pd.to_datetime(df1.purchase_date)
    df1['purchase_date_month_diff'] = df1['today_purchase_date'].dt.total_seconds()/(3600*24*30) - df1.month_lag


    gb = df1.groupby('card_id')['purchase_date_month_diff'].apply(sorted).apply(np.diff)

    try:
        mean = gb.apply(np.mean).rename("purchase_date_month_diff"+'_mean')
        median = gb.apply(np.median).rename("purchase_date_month_diff"+'_median')
        std = gb.apply(np.std).rename("purchase_date_month_diff"+'_std')
        max1 = gb.apply(np.max).rename("purchase_date_month_diff"+'_max')
        min1 = gb.apply(np.min).rename("purchase_date_month_diff"+'_min')
        sum1 = gb.apply(np.sum).rename("purchase_date_month_diff"+'_sum')
    except:
        flag = 1
        
     #=============================== Appending into One File ================

    #pd.DataFrame(df_features['old_hist_count']).to_csv("appended.csv", index=True)

    List = [df_features['hist_count'], df_authorized_flag_count, df_authorized_flag_fraction, df_category1_count,df_category1_fraction,
               df_category2_count, df_category2_fraction, df_category3_count, df_category3_fraction,
               df_city_id_count, df_city_id_fraction, df_merchant_category_id_count, df_merchant_category_id_fraction,
               df_merchant_id_count, df_merchant_id_fraction, df_state_id_count, df_state_id_fraction,
               df_subsector_id_count, df_subsector_id_fraction, Min, Max, Median, Std, Skew, Mad, Sum]
               
    if(flag !=1):
        List = List+[mean, median, std, max1, min1, sum1]    

    df_concat = pd.concat(List, axis = 1, ignore_index=False)
    df_concat.columns = prefix_string + np.array(df_concat.columns)

    ##df_concat.to_csv("appended.csv")

    for i in range(1):
        # datetime features
        df1['purchase_date'] = pd.to_datetime(df1['purchase_date'])
        df1['month'] = df1['purchase_date'].dt.month
        df1['day'] = df1['purchase_date'].dt.day
        df1['hour'] = df1['purchase_date'].dt.hour
        df1['weekofyear'] = df1['purchase_date'].dt.weekofyear
        df1['weekday'] = df1['purchase_date'].dt.weekday
        df1['weekend'] = (df1['purchase_date'].dt.weekday >=5).astype(int)

        # additional features
        df1['price'] = df1['purchase_amount'] / df1['installments']
        
    #Christmas : December 25 2017
        df1['Christmas_Day_2017']=(pd.to_datetime('2017-12-25')-df1['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
        #Mothers Day: May 14 2017
        df1['Mothers_Day_2017']=(pd.to_datetime('2017-06-04')-df1['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
        #fathers day: August 13 2017
        df1['fathers_day_2017']=(pd.to_datetime('2017-08-13')-df1['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
        #Childrens day: October 12 2017
        df1['Children_day_2017']=(pd.to_datetime('2017-10-12')-df1['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
        #Valentine's Day : 12th June, 2017
        df1['Valentine_Day_2017']=(pd.to_datetime('2017-06-12')-df1['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)
        #Black Friday : 24th November 2017
        df1['Black_Friday_2017']=(pd.to_datetime('2017-11-24') - df1['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

        #2018
        #Mothers Day: May 13 2018
        df1['Mothers_Day_2018']=(pd.to_datetime('2018-05-13')-df1['purchase_date']).dt.days.apply(lambda x: x if x > 0 and x < 100 else 0)

        df1['month_diff'] = ((datetime.datetime.today() - df1['purchase_date']).dt.days)//30
        df1['month_diff'] += df1['month_lag']

        # additional features
        df1['duration'] = df1['purchase_amount']*df1['month_diff']
        df1['amount_month_ratio'] = df1['purchase_amount']/df1['month_diff']

    col_seas = ['month', 'hour', 'weekofyear', 'weekday', 'day']
    aggs = {}
    
    for col in col_seas:
        aggs[col] = ['nunique', 'mean', 'min', 'max']
        
    for i in range(1):
        aggs['purchase_date'] = ['max','min']
        aggs['weekend'] = ['mean']
        aggs['month'] = ['mean', 'min', 'max']
        aggs['weekday'] = ['mean', 'min', 'max']
        aggs['price'] = ['mean','max','min','var']
        aggs['Christmas_Day_2017'] = ['mean']
        aggs['Children_day_2017'] = ['mean']
        aggs['Black_Friday_2017'] = ['mean']
        aggs['Mothers_Day_2018'] = ['mean']
        aggs['duration']=['mean','min','max','var','skew']
        aggs['amount_month_ratio']=['mean','min','max','var','skew']


    df_temp = df1.groupby('card_id').agg(aggs)

    # change column name
    df_temp.columns = pd.Index([e[0] + "_" + e[1] for e in df_temp.columns.tolist()])
    df_temp.columns = [prefix_string+ c for c in df_temp.columns]

    if(prefix_string =='old_'):
        df_temp['old_purchase_date_diff'] = (df_temp['old_purchase_date_max']-df_temp['old_purchase_date_min']).dt.days
        #df_temp['hist_purchase_date_average'] = df_temp['hist_purchase_date_diff']/df_temp['hist_card_id_size']
        df_temp['old_purchase_date_uptonow'] = (datetime.datetime.today()-df_temp['old_purchase_date_max']).dt.days
        df_temp['old_purchase_date_uptomin'] = (datetime.datetime.today()-df_temp['old_purchase_date_min']).dt.days

    if(prefix_string =='new_'):
        df_temp['new_purchase_date_diff'] = (df_temp['new_purchase_date_max']-df_temp['new_purchase_date_min']).dt.days
        #df_temp['hist_purchase_date_average'] = df_temp['hist_purchase_date_diff']/df_temp['hist_card_id_size']
        df_temp['new_purchase_date_uptonow'] = (datetime.datetime.today()-df_temp['new_purchase_date_max']).dt.days
        df_temp['new_purchase_date_uptomin'] = (datetime.datetime.today()-df_temp['new_purchase_date_min']).dt.days

    if(prefix_string =='old_new_'):
        df_temp['old_new_purchase_date_diff'] = (df_temp['old_new_purchase_date_max']-df_temp['old_new_purchase_date_min']).dt.days
        #df_temp['hist_purchase_date_average'] = df_temp['hist_purchase_date_diff']/df_temp['hist_card_id_size']
        df_temp['old_new_purchase_date_uptonow'] = (datetime.datetime.today()-df_temp['old_new_purchase_date_max']).dt.days
        df_temp['old_new_purchase_date_uptomin'] = (datetime.datetime.today()-df_temp['old_new_purchase_date_min']).dt.days


    df_concat_new = pd.concat([df_concat,df_temp], axis = 1, ignore_index=False)

    return df_concat_new

In [ ]:
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
new_transactions = pd.read_csv(new_transactions_path)#, nrows = 100000)
hist_transactions = pd.read_csv(historical_transactions_path)#, nrows = 100000)

In [ ]:
processed_train = featurize_train_test(train)
processed_test = featurize_train_test(test)
df_new = Featurized(new_transactions, 'new_')
df_hist = Featurized(new_transactions, 'hist_')
df_new_hist = pd.concat([df_new,df_hist], axis = 1, ignore_index=False)
train_df = pd.merge(df_new_hist, train , on ='card_id')
test_df = pd.merge(df_new_hist, test , on ='card_id')
train_df.to_csv("train.csv")
test_df.to_csv("test.csv")